In [9]:
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import getpass
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import arxiv

In [10]:
### getting relevant paper from arxiv
from langchain_community.document_loaders import ArxivLoader

query = "burst rate and morphological evolution of the periodically repeating FRB 20180916B"
# query = "The Impact of Positive AGN Feedback on the Properties of Galaxies in a Semi-Analytic Model of Galaxy Formation"

# Load relevant papers from arXiv
arxiv_docs = ArxivLoader(query=query, load_max_docs=3).load()  # Loads up to 3 papers matching the query

In [11]:
arxiv_docs[0]

Document(metadata={'Published': '2023-07-11', 'Title': 'A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B', 'Authors': 'Ketan R. Sand, Daniela Breitman, Daniele Michilli, Victoria M. Kaspi, Pragya Chawla, Emmanuel Fonseca, Ryan Mckinven, Kenzie Nimmo, Ziggy Pleunis, Kaitlyn Shin, Bridget C. Andersen, Mohit Bhardwaj, P. J. Boyle, Charanjot Brar, Tomas Cassanelli, Amanda M. Cook, Alice P. Curtin, Fengqiu Adam Dong, Gwendolyn M. Eadie, B. M. Gaensler, Jane Kaczmarek, Adam Lanman, Calvin Leung, Kiyoshi W. Masui, Mubdi Rahman, Ayush Pandhi, Aaron B. Pearlman, Emily Petroff, Masoud Rafiei-Ravandi, Paul Scholz, Vishwangi Shah, Kendrick Smith, Ingrid Stairs, David C. Stenning', 'Summary': 'FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day\nperiodicity in its activity. In this study, we present morphological properties\nof 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021\nDecember. We recorded raw 

In [12]:
# Extract the title, abstract, and figure descriptions from arxiv_docs and store them as tuples
def extract_paper_info(arxiv_docs):
    paper_info = []
    
    for doc in arxiv_docs:
        title = doc.metadata.get("Title", "No Title")
        abstract = doc.metadata.get("Summary", "No Abstract")
        
        # Extract figure descriptions
        image_descriptions = extract_image_descriptions_from_paper(doc)
        
        # Store title, abstract, and image descriptions as a tuple
        paper_info.append((title, abstract, image_descriptions))
    
    return paper_info

# Function to extract figure descriptions from a paper
def extract_image_descriptions_from_paper(doc):
    image_descriptions = []
    
    # Get the page content of the document
    content = doc.page_content
    
    # Look for text related to figures, e.g., containing "Figure" or "图" keywords
    lines = content.split("\n")
    for line in lines:
        if "Figure" in line or "图" in line:  # Adjust based on the language of the document
            image_descriptions.append(line)
    
    return image_descriptions

# Function to format and display all information of a single paper, ensuring readability
def display_full_paper_info(paper_info):
    title, abstract, image_descriptions = paper_info
    print(f"\nTitle:\n{title}\n")
    print(f"Abstract:\n{abstract}\n")
    
    if image_descriptions:
        print("Image Descriptions:")
        for description in image_descriptions:
            print(f"- {description}")
    else:
        print("No image descriptions found.")
    
    print("-" * 80)

# Function to find and display all information of a specific paper by index
def display_paper_info_by_index(paper_info_list, index):
    if 0 <= index < len(paper_info_list):
        display_full_paper_info(paper_info_list[index])
    else:
        print("Invalid index.")

# Function to find and display all information of a specific paper by title keyword
def display_paper_info_by_keyword(paper_info_list, keyword):
    for title, abstract, image_descriptions in paper_info_list:
        if keyword.lower() in title.lower():
            display_full_paper_info((title, abstract, image_descriptions))
            return
    print(f"No document found with keyword: {keyword}")

# Function to get image descriptions of a specific paper by index
def get_image_descriptions_by_index(paper_info_list, index):
    if 0 <= index < len(paper_info_list):
        _, _, image_descriptions = paper_info_list[index]
        return image_descriptions
    else:
        print("Invalid index.")
        return None

# Function to get image descriptions by a specific keyword within the descriptions
def get_image_descriptions_by_keyword(paper_info_list, keyword):
    matching_descriptions = []
    
    for _, _, image_descriptions in paper_info_list:
        for description in image_descriptions:
            if keyword.lower() in description.lower():
                matching_descriptions.append(description)
    
    if matching_descriptions:
        return matching_descriptions
    else:
        print(f"No image descriptions found with keyword: {keyword}")
        return None

# Function to get image descriptions with document info by keyword
def get_image_descriptions_with_doc_info(paper_info_list, keyword):
    matching_descriptions_with_docs = []
    
    for i, (title, _, image_descriptions) in enumerate(paper_info_list):
        for description in image_descriptions:
            if keyword.lower() in description.lower():
                matching_descriptions_with_docs.append((f"Document {i + 1}: {title}", description))
    
    if matching_descriptions_with_docs:
        return matching_descriptions_with_docs
    else:
        print(f"No image descriptions found with keyword: {keyword}")
        return None



In [13]:
# Extract paper information from arxiv_docs
paper_info_list = extract_paper_info(arxiv_docs)
paper_info_list[0]


('A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B',
 'FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day\nperiodicity in its activity. In this study, we present morphological properties\nof 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021\nDecember. We recorded raw voltage data for 45 of these bursts, enabling\nmicroseconds time resolution in some cases. We studied variation of\nspectro-temporal properties with time and activity phase. We find that the\nvariation in Dispersion Measure (DM) is $\\lesssim$1 pc cm$^{-3}$ and that there\nis burst-to-burst variation in scattering time estimates ranging from\n$\\sim$0.16 to over 2 ms, with no discernible trend with activity phase for\neither property. Furthermore, we find no DM and scattering variability\ncorresponding to the recent change in rotation measure from the source, which\nhas implications for the immediate environment of the source. 

In [14]:
# Example: Display the full information of the first document
display_paper_info_by_index(paper_info_list, 0)



Title:
A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B

Abstract:
FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day
periodicity in its activity. In this study, we present morphological properties
of 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021
December. We recorded raw voltage data for 45 of these bursts, enabling
microseconds time resolution in some cases. We studied variation of
spectro-temporal properties with time and activity phase. We find that the
variation in Dispersion Measure (DM) is $\lesssim$1 pc cm$^{-3}$ and that there
is burst-to-burst variation in scattering time estimates ranging from
$\sim$0.16 to over 2 ms, with no discernible trend with activity phase for
either property. Furthermore, we find no DM and scattering variability
corresponding to the recent change in rotation measure from the source, which
has implications for the immediate environment of the source. 

In [15]:

# Example: Display the full information of a document by title keyword
display_paper_info_by_keyword(paper_info_list, "FRB 20180916B")


Title:
A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B

Abstract:
FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day
periodicity in its activity. In this study, we present morphological properties
of 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021
December. We recorded raw voltage data for 45 of these bursts, enabling
microseconds time resolution in some cases. We studied variation of
spectro-temporal properties with time and activity phase. We find that the
variation in Dispersion Measure (DM) is $\lesssim$1 pc cm$^{-3}$ and that there
is burst-to-burst variation in scattering time estimates ranging from
$\sim$0.16 to over 2 ms, with no discernible trend with activity phase for
either property. Furthermore, we find no DM and scattering variability
corresponding to the recent change in rotation measure from the source, which
has implications for the immediate environment of the source. 

In [16]:
# Example: Get image descriptions of the first document
image_descriptions = get_image_descriptions_by_index(paper_info_list, 0)
if image_descriptions:
    print("Image Descriptions for Document 1:")
    for description in image_descriptions:
        print(f"- {description}")

Image Descriptions for Document 1:
- Figures 8 and 9 in the Appendix show our burst sample with baseband data, with Table 1 listing their measured
- properties. Figure 10 show the new detections with only intensity data with measured morphological properties listed
- The pipeline has three main parts. See Figure 1 for a graphical representation of the pipeline workflow.
- Figure 1. Flowchart showing the Baseband Morphology Pipeline. Path on the left involves extracting the Stokes Intensity
- is the period derivative. We then calculate χ2 values again for each period and period derivative combination. Figure
- Figure 2. χ2 distribution of different period (x-axis) and period derivative (y-axis) combinations. The red cross shows the
- Figure 3. Distribution of burst counts for FRB 20180916B . The left panel shows the number of bursts detected during each
- We have had 74 activity cycles since the source’s discovery in 2018 September up to the end of 2021. Figure 3 (left)
- & Stephens 198

In [17]:
# Example: Get image descriptions containing a specific keyword
keyword_descriptions = get_image_descriptions_by_keyword(paper_info_list, "morphological")
if keyword_descriptions:
    print("\nImage Descriptions matching 'morphological':")
    for description in keyword_descriptions:
        print(f"- {description}")


Image Descriptions matching 'morphological':
- properties. Figure 10 show the new detections with only intensity data with measured morphological properties listed


# no RAG


In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)


docs = text_splitter.split_documents(arxiv_docs)

In [19]:
len(docs)

653

In [20]:
pdf_data = []
for doc in arxiv_docs:
    texts = text_splitter.create_documents([doc.page_content])
    pdf_data.append(texts)

In [21]:
paper_info_list

[('A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B',
  'FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day\nperiodicity in its activity. In this study, we present morphological properties\nof 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021\nDecember. We recorded raw voltage data for 45 of these bursts, enabling\nmicroseconds time resolution in some cases. We studied variation of\nspectro-temporal properties with time and activity phase. We find that the\nvariation in Dispersion Measure (DM) is $\\lesssim$1 pc cm$^{-3}$ and that there\nis burst-to-burst variation in scattering time estimates ranging from\n$\\sim$0.16 to over 2 ms, with no discernible trend with activity phase for\neither property. Furthermore, we find no DM and scattering variability\ncorresponding to the recent change in rotation measure from the source, which\nhas implications for the immediate environment of the source

In [22]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(docs,embeddings) ## here we can also use  db = Chroma.from_documents(pdf_data[0],embeddings)

C:\Users\10236\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\10236\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\10236\AppData\Roaming\Python\Python311\site-packages\transformers\models\bert\modeling_bert.py:439: U

In [23]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [24]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

qa = RetrievalQA.from_chain_type(llm=llm, 
                                chain_type="stuff",
                                retriever=db.as_retriever())

In [25]:
question = "what was the total exposure on the source?"
result = qa({"query": question})

print(result)

C:\Users\10236\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


{'query': 'what was the total exposure on the source?', 'result': "The text provides two different exposure times:\n\n* **223 hours:** This refers to the total exposure time across 38 activity cycles using two specific observing strategies.\n* **201 hours:** This refers to the total exposure time within a specific time interval where 94 bursts were detected. \n\nIt's unclear whether these time intervals overlap or refer to completely separate observations. Therefore, I cannot definitively say what the total exposure on the source was. \n"}


In [26]:
question = "can you describe figure 1"
result = qa({"query": question})

print(result)

{'query': 'can you describe figure 1', 'result': 'I do not have access to external data, including figures. Therefore, I cannot describe figure 1. \n'}


In [52]:
question = "can you summerize the context?"
result = qa({"query": question})

print(result)

{'query': 'can you summerize the context?', 'result': 'This excerpt from a scientific paper discusses observations of a repeating Fast Radio Burst (FRB) source, likely FRB 20180916B. The researchers used data from the LOFAR and CHIME telescopes to study the source\'s activity and properties. \n\nHere are some key takeaways:\n\n* **Activity Window:** The bursts from this FRB are not always detectable, but appear in a limited "activity window." The researchers refined the window\'s characteristics using new data.\n* **Rotation Measure (RM) Variations:** The researchers found that the RM, which indicates magnetic field properties along the line of sight, varies over time. These variations happen on longer timescales than the activity window.\n* **Magnetized Screen:** The RM variations suggest the presence of a magnetized screen, like a supernova remnant or pulsar wind nebula, between us and the FRB source. However, no such object has been directly observed.\n* **Burst Analysis:** The rese

# With RAG

In [27]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [28]:
def augment_prompt(query: str):
    # get top 3 context slices
    results = vectorstores.similarity_search(query, k=3)
    source_knowledge = "\n".join([x.page_content for x in results])

    # construct prompt
    augment_prompt = f"""Using the contexts below, answer the query.

    contexts:
    {source_knowledge}

    query: {query}"""
    
    return augment_prompt


In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Adjust as needed
    chunk_overlap=50  # Adjust as needed
)

# Function to combine title, abstract, and image descriptions into a single text block
def combine_paper_info(paper_info):
    title, abstract, image_descriptions = paper_info
    combined_text = f"Title: {title}\n\nAbstract: {abstract}\n\n"
    
    if image_descriptions:
        combined_text += "Image Descriptions:\n" + "\n".join(image_descriptions)
    
    return combined_text

# Process each paper in paper_info_list and split the content using text_splitter
split_documents = []

for paper_info in paper_info_list:
    # Combine the paper info into a single text block
    combined_text = combine_paper_info(paper_info)
    
    # Split the combined text into chunks
    text_chunks = text_splitter.create_documents([combined_text])
    
    # Store the split chunks
    split_documents.extend(text_chunks)

# Example: Print the split chunks for the first paper
for idx, chunk in enumerate(split_documents):
    print(f"Chunk {idx + 1}:\n{chunk.page_content}\n{'-' * 80}\n")

Chunk 1:
Title: A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B
--------------------------------------------------------------------------------

Chunk 2:
Abstract: FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day
periodicity in its activity. In this study, we present morphological properties
of 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021
December. We recorded raw voltage data for 45 of these bursts, enabling
microseconds time resolution in some cases. We studied variation of
spectro-temporal properties with time and activity phase. We find that the
--------------------------------------------------------------------------------

Chunk 3:
variation in Dispersion Measure (DM) is $\lesssim$1 pc cm$^{-3}$ and that there
is burst-to-burst variation in scattering time estimates ranging from
$\sim$0.16 to over 2 ms, with no discernible trend with activity phase for
either property. Furth

In [31]:
vectorstores = Chroma.from_documents(documents = docs, embedding = embeddings, collection_name = "Googai_embed")

In [32]:
query = " can you describe Figure 1"
result = vectorstores.similarity_search(query, k=2)

In [33]:
result

[Document(metadata={'Authors': 'R. Mckinven, B. M. Gaensler, D. Michilli, K. Masui, V. M. Kaspi, M. Bhardwaj, T. Cassanelli, P. Chawla, F. Adam Dong, E. Fonseca, C. Leung, D. Z. Li, C. Ng, C. Patel, E. Petroff, A. B. Pearlman, Z. Pleunis, M. Rafiei-Ravandi, M. Rahman, K. R. Sand, K. Shin, P. Scholz, I. H. Stairs, K. Smith, J. Su, S. Tendulkar', 'Published': '2022-05-18', 'Summary': "Fast radio burst (FRB) source 20180916B exhibits a 16.33-day periodicity in\nits burst activity. It is as of yet unclear what proposed mechanism produces\nthe activity, but polarization information is a key diagnostic. Here, we report\non the polarization properties of 44 bursts from FRB 20180916B detected between\n2018 December and 2021 December by CHIME/FRB, the FRB project on the Canadian\nHydrogen Intensity Mapping Experiment the Canadian Hydrogen Intensity Mapping\nExperiment. In contrast to previous observations, we find significant\nvariations in the Faraday rotation measure (RM) of FRB 20180916B. Ov

In [35]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    contexts:
    Figure 2 displays the individual burst properties of the 44-
Figure A.1 displays FDFs of our 44-burst sample. Each FDF
The RM evolution reported in Figure 2 appears to have
Figure 1: Waterfall plots in Stokes I of bursts from repeating source FRB 20180916B in chronological order and dedispersed
Figure 1: (Continued) Waterfall plots of bursts from repeating source FRB 20180916B.
Figure 2: Burst properties as a function of time for baseband data recorded from FRB 20180916B, displaying ionospheric
Figure B.1: (Continued) The results of QU-fitting (left) and RM-synthesis (right) for individual bursts from repeating source
Figure B.1: (Continued) The results of QU-fitting (left) and RM-synthesis (right) for individual bursts from repeating source
Figure B.1: (Continued) The results of QU-fitting (left) and RM-synthesis (right) for individual bursts from repeating source
Figure B.1: (Continued) The results of QU-fitting (left) an

In [39]:
messages = [
    SystemMessage(content="You are a helpful message"),
    HumanMessage(content=query)
]

In [40]:
# create prompt

prompt = HumanMessage(
    content = augment_prompt(query)
)

messages.append(prompt)

res = llm(messages)

print(res.content)

C:\Users\10236\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Figure 1 is a waterfall plot that depicts bursts from the repeating source FRB 20180916B in chronological order. The bursts are shown in Stokes I and have been dedispersed. 



# for previous question

In [41]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Adjust as needed
    chunk_overlap=50  # Adjust as needed
)

# Function to extract figure descriptions and the remaining text
def extract_figure_descriptions_and_remaining_text(page_content):
    lines = page_content.split("\n")
    figure_descriptions = []
    remaining_text_lines = []
    
    # Iterate through the lines to separate figure descriptions from the rest of the text
    for line in lines:
        if "Figure" in line or "图" in line:  # Adjust based on the language of the document
            figure_descriptions.append(line)
        else:
            remaining_text_lines.append(line)
    
    # Combine the remaining text back into a single string
    remaining_text = "\n".join(remaining_text_lines).strip()
    
    return figure_descriptions, remaining_text

# Function to extract title, abstract, figure descriptions, and remaining text from arxiv_docs
def extract_paper_info_with_remaining_text(arxiv_docs):
    paper_info = []
    
    for doc in arxiv_docs:
        title = doc.metadata.get("Title", "No Title")
        abstract = doc.metadata.get("Summary", "No Abstract")
        page_content = doc.page_content if doc.page_content else ""

        # Extract figure descriptions and remaining text
        figure_descriptions, remaining_text = extract_figure_descriptions_and_remaining_text(page_content)
        
        # Ensure all four values are returned, even if some are empty
        paper_info.append((title, abstract, figure_descriptions, remaining_text))
    
    return paper_info

# Function to combine title, abstract, figure descriptions, and remaining text into a single text block
def combine_paper_info_with_text(paper_info):
    title, abstract, image_descriptions, remaining_text = paper_info
    
    combined_text = f"Title: {title}\n\nAbstract: {abstract}\n\n"
    
    if image_descriptions:
        combined_text += "Image Descriptions:\n" + "\n".join(image_descriptions) + "\n\n"
    
    if remaining_text:
        combined_text += f"Remaining Text:\n{remaining_text}"
    
    return combined_text

# Process each paper in paper_info_list and split the content using text_splitter
def process_and_split_papers(paper_info_list):
    split_documents = []

    for paper_info in paper_info_list:
        # Combine the paper info and remaining text into a single text block
        combined_text = combine_paper_info_with_text(paper_info)
        
        # Split the combined text into chunks
        text_chunks = text_splitter.create_documents([combined_text])
        
        # Store the split chunks
        split_documents.extend(text_chunks)

    return split_documents

# Example: Extract paper info from arxiv_docs and split text into chunks
paper_info_list = extract_paper_info_with_remaining_text(arxiv_docs)
split_documents = process_and_split_papers(paper_info_list)

# Print the split chunks for the first paper
for idx, chunk in enumerate(split_documents):
    print(f"Chunk {idx + 1}:\n{chunk.page_content}\n{'-' * 80}\n")



Chunk 1:
Title: A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B
--------------------------------------------------------------------------------

Chunk 2:
Abstract: FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day
periodicity in its activity. In this study, we present morphological properties
of 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021
December. We recorded raw voltage data for 45 of these bursts, enabling
microseconds time resolution in some cases. We studied variation of
spectro-temporal properties with time and activity phase. We find that the
--------------------------------------------------------------------------------

Chunk 3:
variation in Dispersion Measure (DM) is $\lesssim$1 pc cm$^{-3}$ and that there
is burst-to-burst variation in scattering time estimates ranging from
$\sim$0.16 to over 2 ms, with no discernible trend with activity phase for
either property. Furth

In [42]:
split_documents

[Document(page_content='Title: A CHIME/FRB study of burst rate and morphological evolution of the periodically repeating FRB 20180916B'),
 Document(page_content='Abstract: FRB 20180916B is a repeating Fast Radio Burst (FRB) with a 16.3-day\nperiodicity in its activity. In this study, we present morphological properties\nof 60 FRB 20180916B bursts detected by CHIME/FRB between 2018 August and 2021\nDecember. We recorded raw voltage data for 45 of these bursts, enabling\nmicroseconds time resolution in some cases. We studied variation of\nspectro-temporal properties with time and activity phase. We find that the'),
 Document(page_content='variation in Dispersion Measure (DM) is $\\lesssim$1 pc cm$^{-3}$ and that there\nis burst-to-burst variation in scattering time estimates ranging from\n$\\sim$0.16 to over 2 ms, with no discernible trend with activity phase for\neither property. Furthermore, we find no DM and scattering variability\ncorresponding to the recent change in rotation measur

In [43]:
query = " what was the total exposure on the source?"
result = vectorstores.similarity_search(query, k=2)
vectorstores = Chroma.from_documents(documents = split_documents, embedding = embeddings, collection_name = "Googai_embed")
messagess = [
    SystemMessage(content="You are a helpful message"),
    HumanMessage(content=query)
]
prompt = HumanMessage(
    content = augment_prompt(query)
)

messagess.append(prompt)

res = llm(messagess)

print(res.content)

The total exposure time on the source was **223 hours**. 



In [51]:
query = " can you summerize the context?"
result = vectorstores.similarity_search(query, k=2)
vectorstores = Chroma.from_documents(documents = split_documents, embedding = embeddings, collection_name = "Googai_embed")
messagess = [
    SystemMessage(content="You are a helpful message"),
    HumanMessage(content=query)
]
prompt = HumanMessage(
    content = augment_prompt(query)
)

messagess.append(prompt)

res = llm(messagess)

print(res.content)

This passage describes an update to a study observing a repeating Fast Radio Burst source. The authors are using data from the LOFAR telescope to better constrain the characteristics of the source's activity. 

Here's a breakdown:

* **Increased Observation Time:** They have 2.5 times more data than a previous study (P21), allowing for stronger conclusions.
* **Activity Window:**  The focus is on determining the duration and timing ("width" and "phase") of periods when the source emits bursts.
* **Visualizations:** The results are presented in several graphs:
    * Constraints on the activity window parameters (comparing current to previous results).
    * Total bursts and observation time over time and by activity phase.
    * Burst rate (detections and upper limits) over time and by activity phase.

Essentially, the authors are using a larger dataset to refine their understanding of when and how often this Fast Radio Burst source is active. 

